In [2]:
import carla
client = carla.Client('localhost', 2000)
client.load_world('Town05') #This may give an error "RuntimeError: time-out of 5000ms while w..." - IGNORE

In [3]:
#get world object and select preferred car
world = client.get_world()
vehicle_bp = world.get_blueprint_library().filter('*model3*')

In [10]:
#confirm only one car
len(vehicle_bp)

1

In [3]:
#get all highway spawn points for town 5
import random
good_roads = [12, 34, 35, 36, 37, 38, 1201, 1236, 2034, 2035, 2343, 2344]
spawn_points = world.get_map().get_spawn_points()
good_spawn_points = []
for point in spawn_points:
    this_waypoint = world.get_map().get_waypoint(point.location,project_to_road=True, lane_type=(carla.LaneType.Driving))
    if this_waypoint.road_id in good_roads:
        good_spawn_points.append(point)

transform = random.choice(good_spawn_points)

In [19]:
print(transform) # location of spawning point

Transform(Location(x=7.056836, y=-207.482941, z=0.300000), Rotation(pitch=0.767256, yaw=179.757080, roll=0.000000))


In [4]:
#spawn the car
vehicle = world.try_spawn_actor(vehicle_bp[0], transform)

In [5]:
print(vehicle.get_transform()) 

Transform(Location(x=69.430626, y=-186.747208, z=0.001650), Rotation(pitch=0.000000, yaw=-0.242950, roll=0.000000))


In [24]:
#example of getting straight direction - values low, likely due to the car being straight
print(vehicle.get_transform().get_forward_vector())

Vector3D(x=0.999928, y=-0.004226, z=-0.011186)


In [55]:
#spin the car a little - 30 degrees right
trans = vehicle.get_transform()
trans.rotation.yaw = trans.rotation.yaw + 30
vehicle.set_transform(trans)

In [26]:
# now get straight vector after spinning 30 degrees right - strangely both x & y changed and none by 30 degree
print(vehicle.get_transform().get_forward_vector())

Vector3D(x=0.868084, y=0.496311, z=-0.010304)


In [34]:
import numpy as np
import math
print("\nResult...",math.degrees(np.arctanh(0.496311/0.868084)))


Result... 37.24749732679996


In [35]:
#spin the car back - 30 degrees
trans = vehicle.get_transform()
trans.rotation.yaw = trans.rotation.yaw - 30
vehicle.set_transform(trans)

In [36]:
print(vehicle.get_transform().get_forward_vector())

Vector3D(x=0.999930, y=-0.004226, z=-0.011083)


In [37]:
#spin the car 90 degrees right
trans = vehicle.get_transform()
trans.rotation.yaw = trans.rotation.yaw + 90
vehicle.set_transform(trans)

In [38]:
print(vehicle.get_transform().get_forward_vector())

Vector3D(x=0.004302, y=0.999991, z=-0.000274)


In [43]:
0.496311/(0.868084+0.496311) * 90

32.738312585431636

In [44]:
#spin the car straight
trans = vehicle.get_transform()
trans.rotation.yaw = trans.rotation.yaw - 90
vehicle.set_transform(trans)

Vector3D(x=0.710117, y=0.704029, z=-0.008788)


In [49]:
0.49/(0.86+0.49) * 90


32.666666666666664

In [58]:
vector = vehicle.get_transform().get_forward_vector()
x = vector.x
y = vector.y
print("Rough angle:", y/(x+y))

Rough angle: 0.36371525400034294


In [59]:
#spin the car back - 30 degrees
trans = vehicle.get_transform()
trans.rotation.yaw = trans.rotation.yaw - 30
vehicle.set_transform(trans)

In [60]:
vector = vehicle.get_transform().get_forward_vector()
x = vector.x
y = vector.y
print("Rough angle:", y/(x+y))

Rough angle: -0.004325520234127645


In [61]:
#spin the car back - 30 degrees
trans = vehicle.get_transform()
trans.rotation.yaw = trans.rotation.yaw - 30
vehicle.set_transform(trans)

In [67]:
vector = vehicle.get_transform().get_forward_vector()
x = vector.x
y = vector.y
print("Rough angle:", y/(abs(x)+abs(y)))

Rough angle: -0.3683309051529466


In [64]:
y

-0.5036982893943787

In [65]:
print(vehicle.get_transform().get_forward_vector())

Vector3D(x=0.863817, y=-0.503698, z=-0.010367)


In [10]:
vehicle.set_autopilot(True)


In [11]:
vehicle.set_autopilot(False)

In [12]:
vehicle.destroy()

True

In [5]:
current_waypoint = world.get_map().get_waypoint(vehicle.get_location(),project_to_road=True, lane_type=(carla.LaneType.Driving | carla.LaneType.Sidewalk))

In [8]:
print(current_waypoint.transform.rotation.yaw - vehicle.get_transform().rotation.yaw)

360.0000304877758


In [7]:
print(vehicle.get_transform().rotation.yaw)

-0.2429504096508026


In [5]:
# trying to get all waypoints
# good road_ids
good_roads = [12, 34, 35, 36, 37, 38, 1201, 1236, 2034, 2035, 2343, 2344]
town_map = world.get_map()
all_waypoint_pairs = town_map.get_topology()
# subset of lane start/end's which belong to good roads
good_lanes = []
for w in all_waypoint_pairs:
    if w[0].road_id in good_roads:
        good_lanes.append(w)
        

In [11]:
# loop through selected lanes - trace each lane
# this is using the spectator
import time
spectator = world.get_spectator()
for lane in good_lanes:
    #set start and end of the lane
    for wp in lane[0].next_until_lane_end(10):
        transform = wp.transform
        #lift above the road by 1m
        transform.location.z = transform.location.z+1
        spectator.set_transform(transform)
        time.sleep(0.01)
        

    



In [22]:
print(good_lanes[0][0])

Waypoint(Transform(Location(x=19.325249, y=-193.534851, z=0.000000), Rotation(pitch=360.000000, yaw=359.757080, roll=0.000000)))
